In [1]:
%load_ext autoreload
%autoreload 2
# os.environ["CUDA_VISIBLE_DEVICES"]="1"
from qlora import *
from collections import defaultdict
import copy
import json
import os
from os.path import exists, join, isdir
from dataclasses import dataclass, field
import sys
from typing import Optional, Dict, Sequence
import numpy as np
from tqdm import tqdm
import logging
import bitsandbytes as bnb
import pandas as pd
import importlib
from packaging import version
from packaging.version import parse
import warnings
from sklearn.metrics.pairwise import manhattan_distances
from torchmetrics.functional.pairwise import pairwise_manhattan_distance as manhattan
from torchmetrics.functional.pairwise import pairwise_cosine_similarity as cossim
import numpy as np

import torch
import transformers
from torch.nn.utils.rnn import pad_sequence
import argparse
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    set_seed,
    Seq2SeqTrainer,
    BitsAndBytesConfig,
    LlamaTokenizer

)
from datasets import load_dataset, Dataset, load_from_disk
import evaluate

from peft import (
    prepare_model_for_kbit_training,
    LoraConfig,
    get_peft_model,
    PeftModel
)
from peft.tuners.lora import LoraLayer
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR

from transformers.modeling_utils import unwrap_model
from transformers.models.auto.modeling_auto import MODEL_FOR_CAUSAL_LM_MAPPING_NAMES
from transformers.utils import is_peft_available
from peft import PeftModel

from multihead_models import MultiHeadPeftModelForCausalLM

In [2]:
argdict = {
  'model_name_or_path' : '/mnt/data/zoo/llama2/llama2-7b-hf/',
  'multihead' : 1,
  'use_auth' : True,
  'output_dir' : '/mnt/data/sonia/ckpts/debug',
  'logging_steps' : 10 ,
  'save_strategy' : 'steps',
  'data_seed' : 42 ,
  'save_steps' : 5 ,
  'save_total_limit' : 40 ,
  'evaluation_strategy' : 'steps' ,
  'eval_dataset_size' : 5 ,
  'max_eval_samples' : 100 ,
  'per_device_eval_batch_size' : 1 ,
  'max_new_tokens' : 60 ,
  'dataloader_num_workers' : 1 ,
  'group_by_length' : True,
  'logging_strategy' : 'steps' ,
  'remove_unused_columns' : False ,
  'do_train' : True ,
  'eval_samples' : True ,
  'do_mmlu_eval' : False ,
  'diversity' : False ,
  'divdist' : 'manhattan' ,
  'lora_r' : 64 ,
  'lora_alpha' : 16 ,
  'lora_modules' : 'all' ,
  'double_quant' : True,
  'quant_type' : 'nf4' ,
  'bf16' : True,
  'bits' : 4 ,
  'warmup_ratio' : 0.03 ,
  'lr_scheduler_type' : 'constant' ,
  'gradient_checkpointing' : True,
  'dataset' : '/mnt/data/sonia/honeygan/llama_format_mar01.dat',
  'source_max_len' : 60 ,
  'target_max_len' : 60 ,
  'per_device_train_batch_size' : 1 ,
  'gradient_accumulation_steps' : 16 ,
  'max_steps' : 60 ,
  'eval_steps' : 1 ,
  'learning_rate' : 0.0002 ,
  'adam_beta2' : 0.999 ,
  'max_grad_norm' : 0.3 ,
  'lora_dropout' : 0.1 ,
  'weight_decay' : 0.0 ,
  'seed' : 0
}

arglist = [f'--{k}={v}' for k,v in argdict.items()]

In [3]:
hfparser = transformers.HfArgumentParser((
    ModelArguments, DataArguments, TrainingArguments, GenerationArguments
))
model_args, data_args, training_args, generation_args  = hfparser.parse_args_into_dataclasses(args=arglist, return_remaining_strings=True)[:-1]
training_args.generation_config = transformers.GenerationConfig(**vars(generation_args))
args = argparse.Namespace(
    **vars(model_args), **vars(data_args), **vars(training_args)
)

In [4]:
checkpoint_dir, completed_training = get_last_checkpoint(args.output_dir)
model, tokenizer = get_accelerate_model(args, checkpoint_dir)
model.config.use_cache = False
print('loaded model')
set_seed(args.seed)

data_module = make_data_module(tokenizer=tokenizer, args=args)

Found a previous checkpoint at: /mnt/data/sonia/ckpts/debug/checkpoint-15
loading base model /mnt/data/zoo/llama2/llama2-7b-hf/...


/mnt/data/sonia/miniconda3/envs/llama/lib/python3.11/site-packages/transformers/modeling_utils.py:2193: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/mnt/data/sonia/miniconda3/envs/llama/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Adding special tokens.
Loading adapters from checkpoint.
loaded model
Splitting train dataset in train and validation according to `eval_dataset_size`


In [40]:
trainerclass = Seq2SeqTrainer
trainer = trainerclass(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    **{k:v for k,v in data_module.items() if k != 'predict_dataset'},
)
class evalSampleCallback(transformers.TrainerCallback):
    def on_evaluate(self, args, state, control, model, **kwargs):
        trainer.model.eval()
        metrics = trainer.predict(test_dataset=data_module['eval_dataset'],metric_key_prefix="predict")
        
        predictions = []
        for i in range(len(metrics.predictions)):
            logit = metrics.predictions[i]
            label = metrics.label_ids[i] #just to see positions where prompt tokens are at
            logit_abcd = logit[label != IGNORE_INDEX]
            toks = np.argmax(logit_abcd, axis=1)
            predictions.append(
                ''.join(trainer.tokenizer.decode(toks, skip_special_tokens=True, clean_up_tokenization_spaces=True))
                )
        
        for pred in predictions:
            print(pred)
    
    
trainer.add_callback(evalSampleCallback)

In [42]:
trainer.evaluate(metric_key_prefix="eval")

( 1008 R2 Entercenter 7601 Service Pack 1 ( visible at IP 108. nobody4.111.11 port 40, offering the service cpe:/a:igor_sysoev:ig.
(Build 6. nobody.9600) server visible at IP 102.179.1381,, port 40, offering the service cpe:/a:ig:internet_information_services:8.5.
( ( at IP 102. nobody1.178.116, port 8000, offering the service cpe:/a:ig:557.6.0.
( 8002 R2 Datacenter 9600 ( visible at IP 18. nobody0.114.12, port 80, offering the service cpe:/a:igor_sysoev:ig.
ology DiskStation Manager (DSM) 6. nobody.2-26526 server visible at IP 113.117.11..111, port 5000, offering the service cpe:/a:igor_s


{'eval_loss': 1.0626579523086548,
 'eval_runtime': 1.0422,
 'eval_samples_per_second': 4.798,
 'eval_steps_per_second': 4.798}

In [41]:
metrics = trainer.predict(test_dataset=data_module['eval_dataset'],metric_key_prefix="predict")

In [39]:
for i in range(len(metrics.predictions)):
    logit = metrics.predictions[i]
    label = metrics.label_ids[i] #just to see positions where prompt tokens are at
    logit_abcd = logit[label != IGNORE_INDEX]
    toks = np.argmax(logit_abcd, axis=1)
    print(
        ''.join(trainer.tokenizer.decode(toks, skip_special_tokens=True, clean_up_tokenization_spaces=True))
          )

( 1008 R2 Entercenter 7601 Service Pack 1 ( visible at IP 108. nobody4.111.11 port 40, offering the service cpe:/a:igor_sysoev:ig.
(Build 6. nobody.9600) server visible at IP 102.179.1381,, port 40, offering the service cpe:/a:ig:internet_information_services:8.5.
( ( at IP 102. nobody1.178.116, port 8000, offering the service cpe:/a:ig:557.6.0.
( 8002 R2 Datacenter 9600 ( visible at IP 18. nobody0.114.12, port 80, offering the service cpe:/a:igor_sysoev:ig.
ology DiskStation Manager (DSM) 6. nobody.2-26526 server visible at IP 113.117.11..111, port 5000, offering the service cpe:/a:igor_s


5